In [2]:
import os
import json
from PIL import Image
import requests
from io import BytesIO

# Paths
base_dir = os.getcwd()  # Base directory of the Jupyter notebook
data_dir = os.path.join(base_dir, "data")  # Path to the data folder
annotations_path = os.path.join(data_dir, "annotations.json")  # Path to annotations.json
images_dir = os.path.join(data_dir, "images")  # Folder to save images

# Ensure images directory exists
os.makedirs(images_dir, exist_ok=True)

# Load annotations
with open(annotations_path, 'r') as f:
    annotations = json.load(f)

# Download images
nr_images = len(annotations['images'])
for i, image in enumerate(annotations['images']):
    file_name = image['file_name']
    url_original = image['flickr_url']
    file_path = os.path.join(images_dir, file_name)

    # Ensure any subdirectories in file_path exist
    subdir = os.path.dirname(file_path)
    os.makedirs(subdir, exist_ok=True)

    # Download and save if not already saved
    if not os.path.isfile(file_path):
        response = requests.get(url_original)
        img = Image.open(BytesIO(response.content))
        if img._getexif():
            img.save(file_path, exif=img.info["exif"])
        else:
            img.save(file_path)

    # Display progress in notebook
    bar_size = 30
    x = int(bar_size * (i + 1) / nr_images)
    print(f"Downloading: [{'=' * x}{'.' * (bar_size - x)}] - {i + 1}/{nr_images}", end='\r')

print("\nDownload complete. Images saved in 'data/images/'.")


Downloading: [==============================] - 1500/1500
Download complete. Images saved in 'data/images/'.


In [5]:
import os
import json
import shutil

# Paths
base_path = "./data/images"  # Path to your batches (e.g., batch_1, batch_2, etc.)
annotations_path = "./data/annotations.json"  # Path to your annotations.json file
combined_folder = "./data/combined/images"  # Folder to store all images with new names

# Ensure the combined folder exists
os.makedirs(combined_folder, exist_ok=True)

# Load annotations.json
with open(annotations_path, "r") as f:
    annotations = json.load(f)

print(f"Processing {len(annotations['images'])} images...")

# Process each image in the annotations file
for image in annotations["images"]:
    image_id = image["id"]  # Unique ID for the image
    file_name = image["file_name"]  # Original file path (e.g., batch_1/000006.jpg)
    source_path = os.path.join(base_path, file_name)  # Full path to the original image
    destination_path = os.path.join(combined_folder, f"{image_id}.jpg")  # New path with ID as name

    # Copy image if it exists
    if os.path.exists(source_path):
        shutil.copy2(source_path, destination_path)
    else:
        print(f"Warning: {source_path} not found!")

print(f"All images have been renamed and moved to {combined_folder}.")

Processing 1500 images...
All images have been renamed and moved to ./data/combined/images.


In [13]:
# Paths
annotations_path = "./data/annotations.json"  # Path to your annotations.json file
images_dir = "./data/images"  # Path where your combined images are stored
output_labels_dir = "./data/labels"  # Output folder for YOLO labels
os.makedirs(output_labels_dir, exist_ok=True)  # Ensure the labels directory exists

# Load the COCO annotations
with open(annotations_path, "r") as f:
    coco_data = json.load(f)

# Create a mapping from category_id to supercategory_id
cat_to_supercat_id = {}
supercat_to_id = {}
supercat_counter = 0

for category in coco_data["categories"]:
    supercat = category["supercategory"]
    if supercat not in supercat_to_id:
        supercat_to_id[supercat] = supercat_counter
        supercat_counter += 1
    cat_to_supercat_id[category["id"]] = supercat_to_id[supercat]

# Convert each annotation to YOLO format using supercategories
for annotation in coco_data["annotations"]:
    image_id = annotation["image_id"]
    category_id = annotation["category_id"]
    supercategory_id = cat_to_supercat_id[category_id]  # Map to supercategory ID
    segmentation = annotation["segmentation"]
    bbox = annotation["bbox"]

    # Find the corresponding image
    image_info = next(img for img in coco_data["images"] if img["id"] == image_id)
    image_width = image_info["width"]
    image_height = image_info["height"]

    # YOLO bounding box (normalized)
    x_center = (bbox[0] + bbox[2] / 2) / image_width
    y_center = (bbox[1] + bbox[3] / 2) / image_height
    width = bbox[2] / image_width
    height = bbox[3] / image_height

    # Flatten segmentation polygon points and normalize
    normalized_polygon = [
        f"{x / image_width:.6f} {y / image_height:.6f}"
        for x, y in zip(segmentation[0][::2], segmentation[0][1::2])
    ]
    polygon_points = " ".join(normalized_polygon)

    # Create the label file
    label_file = os.path.join(output_labels_dir, f"{image_id}.txt")
    with open(label_file, "a") as label:
        label.write(f"{supercategory_id} {x_center} {y_center} {width} {height} {polygon_points}\n")

print(f"YOLO format labels with supercategories saved in {output_labels_dir}.")


YOLO format labels with supercategories saved in ./data/labels.


In [18]:
import shutil
import random


# Paths
output_train_dir = "./data/train"
output_val_dir = "./data/val"
images_dir = "./data//images"  # Path where your combined images are stored
output_labels_dir = "./data//labels"  # Output folder for YOLO labels

train_images_dir = os.path.join(output_train_dir, "images")
train_labels_dir = os.path.join(output_train_dir, "labels")
val_images_dir = os.path.join(output_val_dir, "images")
val_labels_dir = os.path.join(output_val_dir, "labels")

os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Get all image file names
image_files = sorted(os.listdir(images_dir))
label_files = sorted(os.listdir(output_labels_dir))

# Shuffle and split
combined = list(zip(image_files, label_files))
random.shuffle(combined)
split_idx = int(0.8 * len(combined))  # 80% train, 20% val

train_set = combined[:split_idx]
val_set = combined[split_idx:]

# Move files to train/val directories
for image_file, label_file in train_set:
    shutil.copy2(os.path.join(images_dir, image_file), train_images_dir)
    shutil.copy2(os.path.join(output_labels_dir, label_file), train_labels_dir)

for image_file, label_file in val_set:
    shutil.copy2(os.path.join(images_dir, image_file), val_images_dir)
    shutil.copy2(os.path.join(output_labels_dir, label_file), val_labels_dir)

print(f"Train and validation sets created in {output_train_dir} and {output_val_dir}.")

Train and validation sets created in ./data/train and ./data/val.


In [24]:
print("Supercategory Mapping:")
print(supercat_to_id)

Supercategory Mapping:
{'Aluminium foil': 0, 'Battery': 1, 'Blister pack': 2, 'Bottle': 3, 'Bottle cap': 4, 'Broken glass': 5, 'Can': 6, 'Carton': 7, 'Cup': 8, 'Food waste': 9, 'Glass jar': 10, 'Lid': 11, 'Other plastic': 12, 'Paper': 13, 'Paper bag': 14, 'Plastic bag & wrapper': 15, 'Plastic container': 16, 'Plastic glooves': 17, 'Plastic utensils': 18, 'Pop tab': 19, 'Rope & strings': 20, 'Scrap metal': 21, 'Shoe': 22, 'Squeezable tube': 23, 'Straw': 24, 'Styrofoam piece': 25, 'Unlabeled litter': 26, 'Cigarette': 27}


In [7]:
import os
train_labels_dir = "./data/train/labels"
val_labels_dir = "./data/val/labels"

# Function to clean label files
def clean_label_files(labels_dir):
    for label_file in os.listdir(labels_dir):
        label_path = os.path.join(labels_dir, label_file)

        # Read the label file
        with open(label_path, "r") as f:
            lines = f.readlines()

        # Process each line
        cleaned_lines = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 5:
                # Retain only the first 5 elements: class_id, x_center, y_center, width, height
                cleaned_lines.append(" ".join(parts[:5]) + "\n")

        # Write cleaned data back to the file
        with open(label_path, "w") as f:
            f.writelines(cleaned_lines)

# Clean labels in train and val directories
clean_label_files(train_labels_dir)
clean_label_files(val_labels_dir)

print("Polygon points removed from all label files.")

Polygon points removed from all label files.


In [26]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

# Train the model
results = model.train(
    data='./data/yolo_data.yaml',  # Path to the data.yaml file
    epochs=50,                # Number of training epochs
    imgsz=640,                # Image size
    batch=64,                 # Batch size
    augment=True,            # Enable data augmentation
    name='yolov8s-seg_EnviroScan_Results'  # Experiment name (results will be saved under this name)
)

New https://pypi.org/project/ultralytics/8.3.68 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.67  Python-3.12.3 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=./data/yolo_data.yaml, epochs=50, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8s-seg_EnviroScan_Results3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, e

train: Scanning C:\Users\shrit\Desktop\Ml_Projects\EnviroScan\EnviroScan\data\train\labels.cache... 1200 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1200/1200 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\shrit\Desktop\Ml_Projects\EnviroScan\EnviroScan\data\val\labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]


Plotting labels to runs\detect\yolov8s-seg_EnviroScan_Results3\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000313, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\yolov8s-seg_EnviroScan_Results3
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50      9.12G     0.7508      1.218     0.9163        126        640: 100%|██████████| 19/19 [00:10<00:00,  1.88it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.73it/s]

                   all        300       1192      0.373      0.155      0.178      0.144



50 epochs completed in 0.187 hours.
Optimizer stripped from runs\detect\yolov8s-seg_EnviroScan_Results3\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\yolov8s-seg_EnviroScan_Results3\weights\best.pt, 6.2MB

Validating runs\detect\yolov8s-seg_EnviroScan_Results3\weights\best.pt...
Ultralytics 8.3.67  Python-3.12.3 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
Model summary (fused): 168 layers, 3,011,108 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.01s/it]


                   all        300       1192      0.435      0.168      0.156      0.123
        Aluminium foil         11         13      0.389      0.308      0.327      0.271
          Blister pack          1          1          1          0     0.0144     0.0144
                Bottle         70         94      0.398      0.564      0.514      0.403
            Bottle cap         47         59      0.373      0.288      0.263      0.175
          Broken glass          4         55          1          0          0          0
                   Can         34         49      0.318      0.633      0.567      0.434
                Carton         42         60       0.28       0.45      0.312      0.242
                   Cup         33         40      0.281        0.4      0.288      0.193
            Food waste          2          2          1          0          0          0
             Glass jar          1          1          0          0     0.0332     0.0298
                   Li

In [15]:
%matplotlib inline

In [35]:
from ultralytics import YOLO
import numpy as np
from tqdm import tqdm

# Load the model
model = YOLO(r'C:\Users\shrit\Desktop\Ml_Projects\EnviroScan\EnviroScan\runs\detect\yolov8s-seg_EnviroScan_Results3\weights\best.pt')  # Replace with your model path

# Run predictions on validation set with confidence threshold
results = model.predict(source='./data/val/images', conf=0.4, imgsz=640)

# Paths
annotations_path = r'C:\Users\shrit\Desktop\Ml_Projects\EnviroScan\EnviroScan\data\val\labels'  # Path to ground truth labels

# Initialize counters
TP, FP, FN = 0, 0, 0

# Process predictions
for i, result in tqdm(enumerate(results), total=len(results)):
    # Load ground truth classes for this image
    image_id = os.path.splitext(os.path.basename(result.path))[0]  # Extract image ID
    label_file = f"{annotations_path}/{image_id}.txt"

    if not os.path.exists(label_file):
        print(f"Missing ground truth for {image_id}")
        continue

    # Parse ground truth classes
    with open(label_file, 'r') as f:
        ground_truth_classes = [int(line.split()[0]) for line in f.readlines()]  # Class IDs

    # Predicted classes
    predicted_classes = result.boxes.cls.cpu().numpy().astype(int)

    # Calculate True Positives and False Positives
    matched_gt = set()  # Track matched ground truth classes
    for pred_class in predicted_classes:
        if pred_class in ground_truth_classes and pred_class not in matched_gt:
            TP += 1  # Correct prediction
            matched_gt.add(pred_class)  # Mark ground truth as matched
        else:
            FP += 1  # Incorrect prediction

    # Calculate False Negatives
    FN += len(set(ground_truth_classes) - matched_gt)

# Print results
print(f"True Positives (TP): {TP}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")


image 1/300 c:\Users\shrit\Desktop\Ml_Projects\EnviroScan\EnviroScan\data\val\images\0.jpg: 640x480 1 Bottle, 12.6ms
image 2/300 c:\Users\shrit\Desktop\Ml_Projects\EnviroScan\EnviroScan\data\val\images\1001.jpg: 480x640 1 Paper, 6.5ms
image 3/300 c:\Users\shrit\Desktop\Ml_Projects\EnviroScan\EnviroScan\data\val\images\1006.jpg: 480x640 1 Styrofoam piece, 6.5ms
image 4/300 c:\Users\shrit\Desktop\Ml_Projects\EnviroScan\EnviroScan\data\val\images\101.jpg: 640x320 (no detections), 6.0ms
image 5/300 c:\Users\shrit\Desktop\Ml_Projects\EnviroScan\EnviroScan\data\val\images\102.jpg: 320x640 1 Bottle, 2 Plastic bag & wrappers, 5.5ms
image 6/300 c:\Users\shrit\Desktop\Ml_Projects\EnviroScan\EnviroScan\data\val\images\1021.jpg: 480x640 1 Plastic bag & wrapper, 7.5ms
image 7/300 c:\Users\shrit\Desktop\Ml_Projects\EnviroScan\EnviroScan\data\val\images\1024.jpg: 480x640 (no detections), 5.5ms
image 8/300 c:\Users\shrit\Desktop\Ml_Projects\EnviroScan\EnviroScan\data\val\images\1029.jpg: 640x480 3 Ca

100%|██████████| 300/300 [00:00<00:00, 5499.33it/s]

True Positives (TP): 193
False Positives (FP): 279
False Negatives (FN): 453
